# <font color = 'blue'> Parameter Space Concept
    method to structure prediction using linear approximation

### <font color = 'green'> **Importing modules**

In [1]:
%matplotlib widget

import warnings
warnings.filterwarnings('ignore')

import os
import time
import numpy as np
import polytope as pc


from psc.lib.g_space import g

from psc.lib.x3Dlinearizazion import linearizenD_EPA
from psc.lib.x3Drepetition import getpolytope_EPA  
from psc.lib.x3Dchecklinearization import checklinear
from psc.lib.x3Dintersection import find_intersection
from psc.lib.x3Dreadwrite import wrtdata



### <font color = 'green'> **Example coordinate**

#### Linearization

In [3]:
# ------------------------------------------------------------------------------------------------
# --->  Generate required information such as atomic coordinate to be solved
#       artificial atomic scattering factors. 'j' fixes the direction of third atomic coordinate
# ------------------------------------------------------------------------------------------------

coordinate = np.array([0.349, 0.362, 0.1615, 0.1615])
f    = [1.0]*len(coordinate)
j    = len(coordinate)-1


# ------------------------------------------------------------------------------------------------
# ---> Apply origin fixing rule. The origin is always fixed at [0, 0, ....]
# ------------------------------------------------------------------------------------------------
l = 1
coordinate = np.sort(coordinate)[::-1]  if (np.sign(g(l, coordinate, f))>0) else np.sort(0.5-coordinate)[::-1]

# ------------------------------------------------------------------------------------------------
# ---> Start to solve given atomic structure using first 4 number of reflections
# ------------------------------------------------------------------------------------------------

h  = 4
info, plist = [], []
IorG='intensity'
print("Assumed coordinate : ", coordinate)

for l in range(1,h+1):
        
    # ===> 1. initilization
    k  = 2*np.pi*l
    gi = np.abs(g(l, coordinate, f))
    amplitudesign = np.sign(g(l, coordinate, f))
    
    # ===> 2. linearization
    normal, distance, boundarypoints = linearizenD_EPA(l, f, gi)
    
    ST = time.time()
    # ===> 3. get all polytope
    p = getpolytope_EPA( l, normal, distance, amplitudesign, IorG, imax=0.5)
    plist.append(p)
    ET = time.time()
    print(f'===> Time taken for RO {l} is {ET-ST}')
    info.append([l, normal, distance])
    
    # ===> 4. check linearization
    checklinear(l, f, gi, normal, distance, j=len(f)-1, n=50, s=1, testiso=True)
###


Assumed coordinate :  [0.3385 0.3385 0.151  0.138 ]
--> Polytope contains complete isosurface. Successful Linearization for RO = 1
===> Time taken for RO 1 is 0.0466303825378418
--> Polytope contains complete isosurface. Successful Linearization for RO = 1
--> Polytope contains complete isosurface. Successful Linearization for RO = 2
===> Time taken for RO 2 is 0.9002726078033447
--> Polytope contains complete isosurface. Successful Linearization for RO = 2
--> Polytope contains complete isosurface. Successful Linearization for RO = 3
===> Time taken for RO 3 is 2.0270841121673584
--> Polytope contains complete isosurface. Successful Linearization for RO = 3
isotype 1
===> Time taken for RO 4 is 5.513397216796875
--> Polytope contains complete isosurface. Successful Linearization for RO = 4


#### Finding intersection

In [4]:
'''
Defining Asym and reduce no. of polytopes in the polytope list for each reflection order
condition: the polytope list must contain first order reflection info.

what if it is starts with other reflection order ? <- yet to solve
'''

# Reducing the polytopes

temp = np.tril(np.ones(shape=(len(f), len(f))) , 0 )
temp = 0.5*np.vstack([[0]*len(normal), temp])
asym = pc.qhull(np.array(temp))

plistr=[]
for i in range(len(plist)):
    r = []
    for ij in plist[i]:
        if ij.intersect(asym):
            r.append(ij)
    plistr.append(r)
    print("===> RO is ",i+1," The len of polytope region before reduction : ",len(plist[i])," after reduction :", len(r), len(plistr))


# finding intersection

solution = []
for inx, ply in enumerate(plistr):
    print("\x1b[0;32m===> intersection for RO : %g"%(inx+1), end="   \x1b[0m")
    if inx==0:
        sf = pc.Region([asym.intersect(ply[0])]) # plistr[inx][0] = ply[0]
        solution.append(sf)
        print(f"len(solution) : {len(sf)} and container len {len(solution)}")
    else:
        tmp= find_intersection(solution[-1], pc.Region(ply))
        solution.append(tmp)
        print(f"len(solution) : {len(tmp)} and container len {len(solution)}")

print("\n===>\x1b[1;31m Variable \x1b[1;3;32msolution\x1b[0m\x1b[1;31m contains the intersection results")

===> RO is  1  The len of polytope region before reduction :  1  after reduction : 1 1
===> RO is  2  The len of polytope region before reduction :  32  after reduction : 10 2
===> RO is  3  The len of polytope region before reduction :  162  after reduction : 30 3
===> RO is  4  The len of polytope region before reduction :  512  after reduction : 70 4
===> intersection for RO : 1   len(solution) : 1 and container len 1
===> intersection for RO : 2   len(solution) : 5 and container len 2
===> intersection for RO : 3   len(solution) : 31 and container len 3
===> intersection for RO : 4   len(solution) : 19 and container len 4

===> Variable solution contains the intersection results


#### Writing dat in h5 file

In [11]:
'''
This section creates hdf file and writes the available information. Current stand takes only one
solution. another for loop is to be added if all RO intersection information is to be written.
'''

# creating file and writing information ! <-- variable names should be regularized, not compatible with readwrite.py

fpath   = os.path.join(os.getcwd())
fn = os.path.join(fpath,'resultfile_%g.h5'%(h))

if os.path.isfile(fn):
    os.remove(fn)
    print(f"===> removed file {fn}")

for jj, i in enumerate(solution[-1]):
    if coordinate in i:
        rc = 0
        xg = np.mean(pc.extreme(i), axis=0)
        volume = pc.volume(i)
        
        extremepnts = pc.extreme(i)
        dmax = np.max(extremepnts, axis=0)
        dmin = np.min(extremepnts, axis=0)
        err  = np.abs(dmax-dmin)/2
        final = i
        
        volAsym = volume
        Lsol=len(solution[-1])
        
        wrtdata(fn, rc, volume, err, final, extremepnts, volAsym, Lsol)


#### Printing centroid of all possible solution region

In [5]:
for jj, i in enumerate(solution[-2]):
    xg = np.mean(pc.extreme(i), axis=0)
    print(xg, jj, coordinate in i)
    if coordinate in i:
        extremepnts = pc.extreme(i)
        dmax = np.max(extremepnts, axis=0)
        dmin = np.min(extremepnts, axis=0)
        err  = np.abs(dmax-dmin)/1
        print(f"\x1b[1;2;32m--> polytope contain coordinate. Predicted \x1b[1;3;34m{xg}\x1b[0;2;32m and assumed \x1b[1;3;34m{coordinate}\x1b[0;2;32m. Error: \x1b[1;3;34m{err}\x1b[0m" )


[0.33082 0.32578 0.17845 0.17059] 0 False
[0.3213  0.29723 0.15614 0.13508] 1 False
[0.32325 0.30309 0.17606 0.13384] 2 False
[0.35425 0.32245 0.18358 0.1723 ] 3 False
[0.3623  0.248   0.14831 0.11161] 4 False
[0.35502 0.28614 0.15396 0.12856] 5 False
[0.48527 0.29222 0.11991 0.0359 ] 6 False
[0.37624 0.2885  0.19606 0.11928] 7 False
[0.37938 0.24009 0.20243 0.08249] 8 False
[0.43361 0.26129 0.25376 0.09429] 9 False
[0.49067 0.31948 0.18035 0.02123] 10 False
[0.32329 0.30321 0.15366 0.12765] 11 False
[0.34974 0.3388  0.15823 0.14137] 12 True
--> polytope contain coordinate. Predicted [0.34974 0.3388  0.15823 0.14137] and assumed [0.3385 0.3385 0.151  0.138 ]. Error: [0.04678 0.02339 0.02981 0.05961]
[0.45417 0.36627 0.12883 0.03942] 13 False
[0.34894 0.30375 0.15235 0.12371] 14 False
[0.47976 0.30669 0.11373 0.0311 ] 15 False
[0.32296 0.30222 0.17755 0.12681] 16 False
[0.35414 0.34027 0.18889 0.14604] 17 False
[0.45919 0.36614 0.18567 0.01478] 18 False
[0.36924 0.29282 0.20038 0.11228]